In [5]:
!pip install "azure-storage-file-datalake>=12.9" "azure-core>=1.29"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.1/264.1 kB 4.4 MB/s eta 0:00:00 0:00:01


In [ ]:
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core.exceptions import ResourceNotFoundError, HttpResponseError

import os
import time

ACCOUNT_NAME = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
ACCOUNT_KEY = os.getenv("AZURE_STORAGE_ACCOUNT_KEY")
FILESYSTEM = os.getenv("AZURE_CONTAINER_NAME")

def dl_client():
    return DataLakeServiceClient(
        account_url=f"https://{ACCOUNT_NAME}.dfs.core.windows.net",  # .dfs para ADLS Gen2 (HNS)
        credential=ACCOUNT_KEY
    )

def clear_directory_preserving_root(max_passes=4, sleep=0.5, ROOT_DIR="GRUPO_6"):
    """
    Vacía ROOT_DIR: borra FILES y luego DIRECTORIOS bottom-up.
    Repite varias pasadas para resolver 409 DirectoryIsNotEmpty por carreras.
    """
    svc = dl_client()
    fs = svc.get_file_system_client(FILESYSTEM)

    # Asegura que exista el directorio raíz
    dirc = fs.get_directory_client(ROOT_DIR)
    try:
        # <- nombre correcto del método
        dirc.get_directory_properties()
    except ResourceNotFoundError:
        print(f"'{ROOT_DIR}' no existe; lo creo vacío.")
        fs.create_directory(ROOT_DIR)
        return

    total_files = total_dirs = 0
    for attempt in range(1, max_passes + 1):
        deleted_files = deleted_dirs = 0

        # 1) Borrar todos los FILES (recursivo)
        for p in fs.get_paths(path=ROOT_DIR, recursive=True):
            if not p.is_directory:
                try:
                    fs.delete_file(p.name)
                    deleted_files += 1
                except ResourceNotFoundError:
                    pass
                except HttpResponseError:
                    # lock transitorio; reintenta en la siguiente pasada
                    pass

        total_files += deleted_files

        # 2) Borrar DIRECTORIOS de lo más profundo a lo menos profundo
        paths = list(fs.get_paths(path=ROOT_DIR, recursive=True))
        paths.sort(key=lambda x: x.name.count("/"), reverse=True)
        for p in paths:
            if p.is_directory:
                try:
                    fs.delete_directory(p.name)  # sin recursive: ya no debería tener hijos
                    deleted_dirs += 1
                except ResourceNotFoundError:
                    pass
                except HttpResponseError:
                    # 409 DirectoryIsNotEmpty; se resolverá en otra pasada
                    pass

        total_dirs += deleted_dirs

        # ¿Quedó vacío?
        any_left = next(fs.get_paths(path=ROOT_DIR, recursive=True), None) is not None
        if not any_left:
            print(f"✅ '{ROOT_DIR}' vacío. Archivos: {total_files}, directorios: {total_dirs}.")
            return

        time.sleep(sleep)

    print(f"⚠️ Tras {max_passes} pasadas aún quedan elementos en '{ROOT_DIR}'. "
          f"Borrados archivos: {total_files}, directorios: {total_dirs}. Re-ejecuta o sube max_passes.")


In [13]:
clear_directory_preserving_root()

✅ 'GRUPO_X' vacío. Archivos: 10, directorios: 10.
